In [ ]:
!pip install pymupdf
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install rank-bm25

In [ ]:
import fitz
import re


pdf_path = "data/constitution_rf.pdf"
def read_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])

    
    full_text = re.sub(r"<.*?>", "", full_text)
    full_text = re.sub(r"[ \t]+", " ", full_text)  
    full_text = full_text.strip().replace('\n', ' ')
    article_pattern = re.compile(r"(Статья \d+)", re.MULTILINE)

    articles = article_pattern.split(full_text)

    article_dict = {}

    for i in range(1, len(articles), 2): 
        article_title = articles[i].strip()
        article_body = articles[i + 1].strip()
        
        points = re.split(r"\d+\.\s", article_body)
        
        points = [f"{idx-1}. {p.strip()}" if idx > 1 else f"{p.strip()}" for idx, p in enumerate(points, start=1) if p.strip()]
        
        article_dict[article_title] = points
    return article_dict

article_dict = dict(read_pdf(pdf_path))
for article, points in list(article_dict.items())[100:105]: 
    print(f"\n{article}:")
    for p in points:
        print(f"{p}")


Статья 101:
1. Совет Федерации избирает из своего состава Председателя Совета Федерации и его заместителей. Государственная Дума избирает из своего состава Председателя Государственной Думы и его заместителей.
2. Председатель Совета Федерации и его заместители, Председатель Государственной Думы и его заместители ведут заседания и ведают внутренним распорядком палаты.
3. Совет Федерации и Государственная Дума образуют комитеты и комиссии, проводят по вопросам своего ведения парламентские слушания.
4. Каждая из палат принимает свой регламент и решает вопросы внутреннего распорядка своей деятельности.
5. Для осуществления контроля за исполнением федерального бюджета Совет Федерации и Государственная Дума образуют Счетную палату, состав и порядок деятельности которой определяются федеральным законом.

Статья 102:
1. К ведению Совета Федерации относятся: а) утверждение изменения границ между субъектами Российской Федерации; б) утверждение указа Президента Российской Федерации о введении во

In [ ]:
import spacy
import string

nlp = spacy.load("ru_core_news_sm")

def clean_and_lemmatize(text):
    text = text.lower().strip().replace("\n", " ") 
    text = re.sub(r"\s+", " ", text) 
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation)) 
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and token.lemma_.strip()]

def create_lemma_dict():
    lemmatized_article_dict = {
        key: [clean_and_lemmatize(p) for p in points if p.strip()] 
        for key, points in article_dict.items()
    }
    return lemmatized_article_dict

lemmatized_article_dict = create_lemma_dict()



In [ ]:
for article, points in list(lemmatized_article_dict.items())[100:105]: 
    print(f"\n{article}:")
    for p in points:
        print(f"{p}")
        
print(lemmatized_article_dict)


Статья 101:
['совет', 'федерация', 'избирать', 'состав', 'председатель', 'совет', 'федерация', 'заместитель', 'государственный', 'дума', 'избирать', 'состав', 'председатель', 'государственный', 'дума', 'заместитель']
['председатель', 'совет', 'федерация', 'заместитель', 'председатель', 'государственный', 'дума', 'заместитель', 'вести', 'заседание', 'ведать', 'внутренний', 'распорядок', 'палата']
['совет', 'федерация', 'государственный', 'дума', 'образовать', 'комитет', 'комиссия', 'проводить', 'вопрос', 'ведение', 'парламентский', 'слушание']
['палата', 'принимать', 'регламент', 'решать', 'вопрос', 'внутренний', 'распорядок', 'деятельность']
['осуществление', 'контроль', 'исполнение', 'федеральный', 'бюджет', 'совет', 'федерация', 'государственный', 'дума', 'образовать', 'счётный', 'палата', 'состав', 'порядок', 'деятельность', 'определяться', 'федеральный', 'закон']

Статья 102:
['ведение', 'совет', 'федерация', 'относиться', 'утверждение', 'изменение', 'граница', 'субъект', 'российс

In [ ]:
from rank_bm25 import BM25Okapi


corpus = [" ".join([" ".join(p) for p in points]) for points in lemmatized_article_dict.values()]
tokenized_corpus = [doc.split() for doc in corpus] 

bm25 = BM25Okapi(tokenized_corpus)

def search_bm25(query, top_n=3):
    query = query.strip()

    if query.lower().startswith("статья"):
        if query in article_dict:
            return [(query, " ".join(article_dict[query]))]
        else:
            return [("Ошибка", f"Статья '{query}' не найдена")]
    
    
    query_tokens = clean_and_lemmatize(query)  
    scores = bm25.get_scores(query_tokens)  
    top_indexes = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n] 
    top_articles = list(lemmatized_article_dict.keys()) 
    
    results = []
    for i in top_indexes:
        article_title = top_articles[i]
        original_text = "\n".join(article_dict[article_title]) 
        results.append((article_title, original_text))
    
    return results

queries = ["Права человека"]
for query in queries:
    results = search_bm25(query)
    print(f"\nЗапрос: {query}")
    for title, text in results:
        print(f"\n{title}:\n{text}")  



Запрос: Права человека

Статья 17:
1. В Российской Федерации признаются и гарантируются права и свободы человека и гражданина согласно общепризнанным принципам и нормам международного права и в соответствии с настоящей Конституцией.
2. Основные права и свободы человека неотчуждаемы и принадлежат каждому от рождения.
3. Осуществление прав и свобод человека и гражданина не должно нарушать права и свободы других лиц.

Статья 2:
Человек, его права и свободы являются высшей ценностью. Признание, соблюдение и защита прав и свобод человека и гражданина - обязанность государства.

Статья 55:
1. Перечисление в Конституции Российской Федерации основных прав и свобод не должно толковаться как отрицание или умаление других общепризнанных прав и свобод человека и гражданина.
2. В Российской Федерации не должны издаваться законы, отменяющие или умаляющие права и свободы человека и гражданина.
3. Права и свободы человека и гражданина могут быть ограничены федеральным законом только в той мере, в как